<a href="https://colab.research.google.com/github/matpg/RL-Agent-for-Unreal-Engine/blob/main/Codigo_StableBaselines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Descarga de Librerias

In [ ]:
!git clone https://github.com/maximecb/gym-minigrid.git
%cd gym-minigrid
!python setup.py install
# Stable Baselines only supports tensorflow 1.x for now
%tensorflow_version 1.x
!apt-get install ffmpeg freeglut3-dev xvfb  # For visualization
!pip install stable-baselines[mpi]==2.10.0
!pip3 install box2d-py

### Importar Librerias

In [ ]:
import gym
import gym_minigrid
import numpy as np
from stable_baselines.common import make_vec_env
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2, DDPG, DQN, A2C, ACER, TRPO
from stable_baselines.common.policies import MlpPolicy, CnnPolicy, MlpLstmPolicy, MlpLnLstmPolicy, CnnLstmPolicy, CnnLnLstmPolicy
# from stable_baselines.deepq.policies import MlpPolicy, CnnPolicy
from stable_baselines.common.evaluation import evaluate_policy

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



### Carga de Entorno Minigrid

In [ ]:
# Eenvironement for learning
# Use a separate environement for evaluation
from gym_minigrid.wrappers import *
env = gym.make('MiniGrid-DoorKey-5x5-v0')
# env = make_vec_env(env, n_envs=2)
env = RGBImgObsWrapper(env) # Get pixel observations
env = ImgObsWrapper(env) # Get rid of the 'mission' field
# env = DirectionObsWrapper(env)
env = DummyVecEnv([lambda: env])
#env = gym.make('MiniGrid-DoorKey-5x5-v0')
print(env.observation_space)
print(env.action_space) # 7
eval_env = env
# vectorized environments allow to easily multiprocess training
# we demonstrate its usefulness in the next examples


Box(40, 40, 3)
Discrete(7)


In [ ]:
%cd /content/gym-minigrid/

/content/gym-minigrid


In [ ]:
# Use a separate environement for evaluation
# eval_env = gym.make('CartPole-v1')

# Random Agent, before training
# mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=100)

# print(f"mean_reward for random agent:{mean_reward:.2f} +/- {std_reward:.2f}")

### Definicion de Modelo del Agente

In [ ]:
# Activate all extensions
kwargs = {'double_q': True, 'prioritized_replay': True, 'policy_kwargs': dict(dueling=True)}

agent_model = A2C('CnnLnLstmPolicy', env, verbose=1, n_steps=10,seed=0)

### Evaluar Agente Ingenuo/Aleatorio

In [ ]:
# Random Agent, before training using property
from stable_baselines.common.evaluation import evaluate_policy
mean_reward, std_reward = evaluate_policy(agent_model, env, n_eval_episodes=100)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

mean_reward:0.00 +/- 0.00


### Entrenar Agente

In [ ]:
agent_model.learn(total_timesteps=2000, log_interval=50)

### Evaluar Agente Entrenado

In [ ]:
# Trained Agent

mean_reward, std_reward = evaluate_policy(agent_model, env, n_eval_episodes=100)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

mean_reward:0.00 +/- 0.00


### Extras

In [ ]:
import gym_minigrid

env = gym.make('MiniGrid-SimpleCrossingEnvUmaze-v0')
for i_episode in range(3):
    obs = env.reset()
    for t_steps in range(100):
        # env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        print(obs)
        if done:
            print("Episode finished after {} timesteps".format(t_steps+1))
            break
env.close()

In [ ]:
!apt update && apt install xvfb
!pip install gym-notebook-wrapper

In [ ]:
import gnwrapper
import gym

def evaluate(model, num_episodes=100):
    """
    Evaluate a RL agent
    :param model: (BaseRLModel object) the RL Agent
    :param num_episodes: (int) number of episodes to evaluate it
    :return: (float) Mean reward for the last num_episodes
    """
    # This function will only work for a single Environment
    # env = gnwrapper.LoopAnimation(model.get_env()) # Start Xvfb
    env = model.get_env()
    all_episode_rewards = []
    for i in range(num_episodes):
        episode_rewards = []
        done = False
        obs = env.reset()
        while not done:
            # _states are only useful when using LSTM policies
            action, _states = model.predict(obs)
            # here, action, rewards and dones are arrays
            # because we are using vectorized env
            obs, reward, done, info = env.step(action)
            # env.render()
            episode_rewards.append(reward)

        all_episode_rewards.append(sum(episode_rewards))
        

    # env.display()
    mean_episode_reward = np.mean(all_episode_rewards)
    print("Mean reward:", mean_episode_reward, "Num episodes:", num_episodes)

    return mean_episode_reward

In [ ]:
# Random Agent, before training using function
mean_reward_before_train = evaluate(agent_model, num_episodes=100)